<a href="https://colab.research.google.com/github/SCCSMARTCODE/Deep-Learning-00/blob/main/AlexNet/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AlexNet**

------
### **Milestone 1: Dataset Preparation**
 - [x] Kaggle Dataset
 - [x] Data Preprocessing
 - [x] Set up DataLoader

---

### **Milestone 2: Data Augmentation Techniques**

   - [x] Horizontal flip (mirror)
   - [x] Random rotation
   - [x] Color jitter
   - [x] Scaling

---

### **Milestone 3: AlexNet Implementation**
---
### **Milestone 4: Regularization Techniques**
 - [x] Weight Decay (L2 Regularization):
 - [x] Dropout:
 - [ ] Early Stopping:
 - [x] Batch Normalization:
---
### **Milestone 5: Model Training**
 - [x] One Cycle Learning Rate
 - [x] Training the Model:
---
### **Milestone 6: Model Evaluation**
 - [x] Accuracy and Loss Curves:
 - [ ] Confusion Matrix:
 - [ ] Precision, Recall, and F1-Score:
---
### **Milestone 7: Hyperparameter Tuning**
 - [x] Optimizer Tuning:
 - [x] Batch Size and Epochs:
 - [x] Augmentation Tuning:

# **```Dataset Preparation```**

- [ ] Fetching data from Kaggle
- [ ] Implement data fetching from **ALEXNET_RESOURCES_PATH**

In [ ]:
# API_KEY_PATH="./drive/MyDrive/Deep_Learning/kaggle.json"
# ALEXNET_RESOURCES_PATH="/content/drive/MyDrive/Deep_Learning/AlexNet"

# !mkdir -p ~/.kaggle
# !cp $API_KEY_PATH ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !pip install kaggle
# !kaggle competitions download -c dogs-vs-cats

# !cp dogs-vs-cats.zip $ALEXNET_RESOURCES_PATH

# !unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [ ]:
ALEXNET_DATASET_PATH = "/content/drive/MyDrive/Deep_Learning/AlexNet/dogs-vs-cats.zip"
TRAIN_DATA_PATH="/content/train.zip"
TEST_DATA_PATH="/content/test1.zip"

!cp $ALEXNET_DATASET_PATH .
!unzip dogs-vs-cats.zip
!unzip $TRAIN_DATA_PATH
!unzip $TEST_DATA_PATH

# **```Importing Dependencies```**

In [ ]:
%pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
from torchvision.io import read_image
import os
import torch
from torchvision import transforms as tt
from torch.utils.data import Dataset, DataLoader, random_split
from typing import Iterator
from torch import nn
from torchsummary import summary
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
import wandb

# **```Loading Dataset```**

In [45]:
class DogVSCatDataset(Dataset):
    def __init__(self, path="/content/train", transform=None):
        self.path = path
        self.files = os.listdir(self.path)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name = self.files[idx]
        image_class = torch.tensor(1 if 'dog' in file_name else 0, dtype=torch.float)
        image_data = read_image(os.path.join(self.path, file_name)) / 255.0

        if self.transform:
            image_data = self.transform(image_data)

        return image_data, image_class


def load_on_device(device: str, dataloader: DataLoader) -> Iterator[torch.Tensor]:
    try:
        for batch_data in dataloader:
            inputs, targets = batch_data
            inputs, targets = inputs.to(device), targets.to(device)
            yield inputs, targets

            del inputs, targets
            torch.cuda.empty_cache()
    except RuntimeError as e:
        print(f"Error loading data on {device}: {e}")

In [53]:
transforms = tt.Compose([
    tt.Resize((227, 227)),
    tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    tt.RandomHorizontalFlip(p=0.5),
    tt.ColorJitter(brightness=0.2, contrast=0.2),
])

dataset = DogVSCatDataset(path="/content/train", transform=transforms)

training_dataset, validation_dataset, testing_dataset = random_split(dataset, [20000, 2500, 2500])


train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
valid_loader = DataLoader(validation_dataset, batch_size=128, shuffle=None, drop_last=True)
test_loader = DataLoader(testing_dataset, batch_size=128, shuffle=None, drop_last=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# **`Model Definition & Initialization`**

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=96),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=384),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=384),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(),

            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(1000, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


network = AlexNet().to(device)
summary(network, input_size=(3,227,227), batch_size=128)
network

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [128, 96, 56, 56]          34,944
              ReLU-2          [128, 96, 56, 56]               0
       BatchNorm2d-3          [128, 96, 56, 56]             192
         MaxPool2d-4          [128, 96, 27, 27]               0
            Conv2d-5         [128, 256, 27, 27]         614,656
              ReLU-6         [128, 256, 27, 27]               0
       BatchNorm2d-7         [128, 256, 27, 27]             512
         MaxPool2d-8         [128, 256, 13, 13]               0
            Conv2d-9         [128, 384, 13, 13]         885,120
             ReLU-10         [128, 384, 13, 13]               0
      BatchNorm2d-11         [128, 384, 13, 13]             768
           Conv2d-12         [128, 384, 13, 13]       1,327,488
             ReLU-13         [128, 384, 13, 13]               0
      BatchNorm2d-14         [128, 384,

AlexNet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Con

# **`Hyperpaarameter & B.P Function Definition `**

In [54]:
EPOCHS=10
LR=1e-5
MAX_LR=1e-3
BATCH_SIZE=128

criterion = nn.BCELoss()
optimizer = Adam(params=network.parameters(), lr=LR, weight_decay=.008)
lr_scheduler = OneCycleLR(optimizer=optimizer, max_lr=MAX_LR, steps_per_epoch=BATCH_SIZE, epochs=EPOCHS)

# **`Training and Metric function definition`**

In [ ]:
@torch.no_grad()
def accuracy(model, data_loader, criterion):
    acc_count = 0
    total_preds = 0
    total_loss = 0.0
    total_batches = len(data_loader)
    data_loader = load_on_device(device, data_loader)

    model.eval()

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        pred = model(data)


        formated_pred = pred.squeeze() >= 0.5
        acc_count += (formated_pred == target).sum().item()
        total_preds += len(target)

        loss = criterion(pred.squeeze(), target.float())
        total_loss += loss.item()

    if total_preds == 0:
        return 0.0, None

    avg_loss = total_loss / total_batches
    accuracy_percentage = (acc_count / total_preds) * 100

    return accuracy_percentage, avg_loss


@torch.no_grad()
def run_inference(model, test_dl, criterion):
    # Get accuracy and loss on test data
    acc_percentage, avg_loss = accuracy(model, test_dl, criterion)

    # Format and print the results
    print("=" * 40)
    print("Inference Results")
    print("=" * 40)
    print(f"Test Accuracy: {acc_percentage:.2f}%")
    print(f"Test Loss: {avg_loss:.4f}")
    print("=" * 40)

In [55]:
wandb.init(project="AlexNet", config={
    "epochs": EPOCHS,
    "learning_rate": optimizer.param_groups[0]['lr'],
})

wandb.watch(network, criterion, log="all")


def train(model, epochs, train_dl, val_dl, criterion, optimizer, lr_scheduler):
    total_train_batches = len(train_dl)
    train_dl = load_on_device(device, train_dl)

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for data, target in train_dl:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            pred = model(data)

            loss = criterion(pred.squeeze(), target)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            epoch_loss = loss.item()

            wandb.log({
                "learning_rate": lr_scheduler.get_last_lr()[0],
                "batch_loss": loss.item(),
                "epoch": epoch + 1
            })
        wandb.log({"train_loss": epoch_loss / total_train_batches})

        ### validation phase
        val_acc, val_loss = accuracy(model, val_dl, criterion)
        wandb.log({"val_loss": val_loss, "val_acc": val_acc, "epoch": epoch + 1})
        torch.save(network.state_dict(), '/content/drive/MyDrive/Deep_Learning/AlexNet/parameter.pth')

    wandb.finish()


# **`Network Training and Inference`**

In [56]:
train(network, EPOCHS, train_loader, valid_loader, criterion, optimizer, lr_scheduler)

batch_loss,▆█▇▄▄▅▆▆▄▆▆▇▃▅█▄▅▃▃▂▅▄▅▃▅▄▄▅▅▁▅▄▆▄▇▇▆▇▄▅
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇██
train_loss,█▁▁▁▁▁▁▁▁▁
val_acc,▃▁▄▇▅█▅▆▂▁
val_loss,▄▁▆▂█▃█▃▇▂
batch_loss,0.34373
epoch,10
learning_rate,0.00038
train_loss,0
val_acc,82.52467


In [57]:
run_inference(network, test_loader, criterion)

Inference Results
Test Accuracy: 83.14%
Test Loss: 0.3608
